In [1]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr

from sklearn.linear_model import LinearRegression
from tqdm import tqdm

from fbprophet.diagnostics import cross_validation
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import numpy as np
from fbprophet import Prophet

import warnings 
warnings.filterwarnings('ignore')

In [2]:
stock = pd.read_csv('Stock_List.csv')
stock

,종목명,종목코드,상장시장
0,삼성전자,5930,KOSPI
1,SK하이닉스,660,KOSPI
2,NAVER,35420,KOSPI
3,카카오,35720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
371,더네이쳐홀딩스,298540,KOSDAQ
372,코엔텍,29960,KOSDAQ
373,원익홀딩스,30530,KOSDAQ
374,웹케시,53580,KOSDAQ


In [3]:
stock['종목코드'] = stock['종목코드'].apply(lambda x : str(x).zfill(6))

In [4]:
price_name = stock.loc[0,'종목코드']
price_name

'005930'

In [5]:
start_date = '20210104'
end_date = '20210827'

In [6]:
price = fdr.DataReader(price_name, start = start_date, end = end_date)[['Close']].reset_index()

In [7]:
work_day = price[['Date']]

In [8]:
work_day

,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08
...,...
159,2021-08-23
160,2021-08-24
161,2021-08-25
162,2021-08-26


In [9]:
for code in tqdm(stock['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    work_day[code] = data['Close']
    

100%|████████████████████████████████████████████████████████████████████████████████| 376/376 [00:32<00:00, 11.62it/s]


In [10]:
work_day

,Date,005930,000660,035420,035720,207940,005935,051910,006400,005380,...,064260,287410,110790,099320,045390,298540,029960,030530,053580,243840
0,2021-01-04,83000,126000,293000,79483,829000,74400,889000,671000,207500,...,4835,4200,25850,31950,6020,19425,9330,6270,35887,49100
1,2021-01-05,83900,130500,292500,78881,832000,75000,893000,686000,209500,...,4820,4240,24750,32950,5980,19700,9190,6190,35634,47900
2,2021-01-06,82200,131000,290000,79383,817000,74300,890000,680000,203000,...,4795,4295,24750,35350,6270,19625,9120,6080,35786,47350
3,2021-01-07,82900,134500,289500,80788,819000,74300,962000,698000,206000,...,4810,4480,24750,38200,6190,19825,9230,6130,35837,48500
4,2021-01-08,88800,138000,312000,87111,837000,77800,999000,739000,246000,...,4755,4300,23700,38300,6240,19750,9190,6170,36495,51000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,2021-08-23,73300,103000,429500,148500,1009000,68500,798000,798000,204000,...,5870,8550,38950,52200,6420,35900,8780,5230,31600,89600
160,2021-08-24,75600,105000,437500,149000,956000,71000,787000,770000,209500,...,6180,8550,39100,53100,6420,36550,8800,5430,32400,87500
161,2021-08-25,75700,103500,430500,151000,963000,70500,799000,776000,209500,...,6300,8370,39500,52800,6470,36550,8860,5520,32250,84500
162,2021-08-26,74600,104000,423500,149500,954000,70100,788000,765000,210000,...,6250,8180,39750,52300,6490,36200,8830,5530,32100,85100


In [11]:
sub = pd.DataFrame()
sub['DateTime'] = pd.date_range('2021-08-23','2021-09-10',freq='B')

In [12]:
sub1 = sub.drop([5,6,7,8,9]).reset_index()

In [13]:
sub2 = sub1.drop(columns = 'index')

In [14]:
sub2

,DateTime
0,2021-08-23
1,2021-08-24
2,2021-08-25
3,2021-08-26
4,2021-08-27
5,2021-09-06
6,2021-09-07
7,2021-09-08
8,2021-09-09
9,2021-09-10


In [15]:
for col in tqdm(work_day.columns[1:].tolist()):
  work_day1 = work_day[['Date', col]]
  work_day1.columns = ['ds','y']

  model = Prophet()
  model.fit(work_day1)

  future = model.make_future_dataframe(periods =365)
  fcst = model.predict(future)
#   print(f'{col} Forecasting....')

  result = fcst.iloc[159:178]
  
  result.rename(columns={'ds':'DateTime'}, inplace = True)
  
  # display(result)
  temp = result.groupby('DateTime')['yhat'].sum().reset_index(name = col)

  sub2 = pd.merge(sub2,temp, on = 'DateTime', how = 'left')
#   display(sub2)

  0%|                                                                                          | 0/376 [00:00<?, ?it/s]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
  0%|▏                                                                                 | 1/376 [00:02<13:55,  2.23s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
  1%|▍                                                                                 | 2/376 [00:03<12:20,  1.98s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
  1%|▋                   

  7%|█████▍                                                                           | 25/376 [00:36<08:32,  1.46s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
  7%|█████▌                                                                           | 26/376 [00:38<08:24,  1.44s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
  7%|█████▊                                                                           | 27/376 [00:39<08:19,  1.43s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
  7%|██████              

 13%|██████████▊                                                                      | 50/376 [01:13<07:54,  1.45s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 14%|██████████▉                                                                      | 51/376 [01:14<07:52,  1.45s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 14%|███████████▏                                                                     | 52/376 [01:15<07:51,  1.46s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 14%|███████████▍        

 20%|████████████████▏                                                                | 75/376 [01:49<07:23,  1.48s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 20%|████████████████▎                                                                | 76/376 [01:50<07:24,  1.48s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 20%|████████████████▌                                                                | 77/376 [01:52<07:36,  1.53s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 21%|████████████████▊   

 27%|█████████████████████▎                                                          | 100/376 [02:29<07:38,  1.66s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 27%|█████████████████████▍                                                          | 101/376 [02:30<07:37,  1.66s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 27%|█████████████████████▋                                                          | 102/376 [02:32<07:34,  1.66s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 27%|████████████████████

...

 73%|██████████████████████████████████████████████████████████▌                     | 275/376 [06:53<02:23,  1.42s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 73%|██████████████████████████████████████████████████████████▋                     | 276/376 [06:54<02:21,  1.41s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 74%|██████████████████████████████████████████████████████████▉                     | 277/376 [06:56<02:19,  1.41s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 74%|████████████████████

 80%|███████████████████████████████████████████████████████████████▊                | 300/376 [07:28<01:46,  1.40s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 80%|████████████████████████████████████████████████████████████████                | 301/376 [07:30<01:45,  1.41s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 80%|████████████████████████████████████████████████████████████████▎               | 302/376 [07:31<01:45,  1.42s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 81%|████████████████████

 86%|█████████████████████████████████████████████████████████████████████▏          | 325/376 [08:04<01:12,  1.42s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 87%|█████████████████████████████████████████████████████████████████████▎          | 326/376 [08:05<01:10,  1.42s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 87%|█████████████████████████████████████████████████████████████████████▌          | 327/376 [08:07<01:09,  1.42s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 87%|████████████████████

 93%|██████████████████████████████████████████████████████████████████████████▍     | 350/376 [08:40<00:37,  1.45s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 93%|██████████████████████████████████████████████████████████████████████████▋     | 351/376 [08:41<00:35,  1.44s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 94%|██████████████████████████████████████████████████████████████████████████▉     | 352/376 [08:43<00:34,  1.44s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 94%|████████████████████

100%|███████████████████████████████████████████████████████████████████████████████▊| 375/376 [09:16<00:01,  1.47s/it]INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
100%|████████████████████████████████████████████████████████████████████████████████| 376/376 [09:18<00:00,  1.48s/it]


In [16]:
sub2

,DateTime,005930,000660,035420,035720,207940,005935,051910,006400,005380,...,064260,287410,110790,099320,045390,298540,029960,030530,053580,243840
0,2021-08-23,76322.060421,105691.642716,436047.406632,144782.059107,9.713299e+05,71085.150916,836227.520844,798022.180849,211775.510117,...,6415.919119,8560.000298,40628.010986,54973.506297,6529.185656,34918.386581,8796.791526,5641.955598,32572.479732,75063.206118
1,2021-08-24,76392.953224,106137.801255,436545.943143,145049.984140,9.725041e+05,71105.370764,835222.137792,802925.644166,212691.798054,...,6404.305047,8622.567680,40541.618264,54809.331287,6531.464660,35090.832285,8769.996787,5599.903125,32656.549119,75037.618904
2,2021-08-25,76004.340115,105050.288659,437417.565234,145704.957615,9.735128e+05,70824.709351,827778.396465,803339.974329,211467.449014,...,6402.464531,8613.771889,40507.434604,54648.254000,6539.422701,35217.247363,8758.016021,5574.645470,32473.211463,75679.999016
3,2021-08-26,75901.133228,104935.983352,437564.281831,145138.284707,9.762139e+05,70698.899574,837572.108108,805606.029663,209979.150936,...,6501.475250,8659.784317,40843.745696,54707.727889,6531.447845,35251.785043,8721.329623,5555.313708,32428.569523,76108.122767
4,2021-08-27,75736.768305,104349.595813,437500.281202,144384.038199,9.805423e+05,70594.817105,838262.442308,809887.175583,209846.777380,...,6374.024164,8691.164087,40834.523343,54353.237930,6575.385165,35219.804757,8681.036535,5534.144803,32314.329919,76695.537897
5,2021-09-06,75077.420525,100784.349249,437379.256595,141273.312241,1.003372e+06,70316.589773,833761.101142,825639.520180,205442.047588,...,6316.035822,9151.878441,42020.713428,55232.363726,6579.803929,35244.346757,8470.608667,5385.617416,31669.346713,80103.260672
6,2021-09-07,75148.313328,101230.507787,437877.793107,141541.237275,1.004547e+06,70336.809620,832755.718090,830542.983496,206358.335525,...,6304.421751,9214.445824,41934.320706,55068.188715,6582.082933,35416.792461,8443.813928,5343.564942,31753.416099,80077.673458
7,2021-09-08,74759.700219,100142.995192,438749.415198,142196.210749,1.005555e+06,70056.148208,825311.976763,830957.313660,205133.986484,...,6302.581235,9205.650033,41900.137046,54907.111428,6590.040974,35543.207539,8431.833162,5318.307287,31570.078443,80720.053570
8,2021-09-09,74656.493332,100028.689885,438896.131795,141629.537841,1.008256e+06,69930.338431,835105.688406,833223.368994,203645.688407,...,6401.591954,9251.662461,42236.448138,54966.585318,6582.066118,35577.745219,8395.146764,5298.975526,31525.436503,81148.177321
9,2021-09-10,74492.128409,99442.302346,438832.131166,140875.291333,1.012585e+06,69826.255962,835796.022606,837504.514913,203513.314850,...,6274.140868,9283.042230,42227.225785,54612.095358,6626.003438,35545.764932,8354.853676,5277.806621,31411.196899,81735.592451


In [17]:
sub3=sub2.sort_index(axis=1,ascending = True)

In [21]:
sub3

,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,...,336260,336370,347860,348150,348210,352820,357780,363280,950130,DateTime
0,25777.856293,31795.723342,60446.774489,169651.903026,90793.351817,17786.718654,56560.870434,83965.562528,105691.642716,662838.366735,...,50002.806615,64430.343722,39155.000359,30199.152777,55741.235337,298386.019125,315536.342192,30780.812611,27985.201056,2021-08-23
1,25999.317065,31817.562548,60330.424835,169410.453573,90424.562138,17805.626435,56427.665746,84040.510966,106137.801255,666932.114789,...,49735.248747,64383.199900,38995.360613,30365.646639,56113.714573,298443.509048,315309.618605,30881.690363,27654.673997,2021-08-24
2,26198.258083,31774.880583,60378.164808,169605.213830,90599.045085,17805.049596,55969.208925,84131.919974,105050.288659,666957.944877,...,50287.061401,64517.068152,39428.198636,30092.154376,56436.901613,297745.112808,311903.657666,30881.024535,27337.297740,2021-08-25
3,26374.746375,31623.244025,60044.979230,169655.675101,90116.175195,17592.467352,56064.794764,83805.864375,104935.983352,667980.435574,...,49824.536724,64807.124198,38981.848924,30171.675963,56142.696145,298659.245417,313504.142362,30833.564954,27711.944386,2021-08-26
4,26413.284116,31445.374576,60009.202461,168418.993826,89520.763376,17511.951718,55863.736464,83778.029695,104349.595813,665198.476362,...,49899.758555,65150.368886,38742.478812,29859.590624,55923.729401,298481.311054,312947.881856,30754.227527,27600.527018,2021-08-27
5,27317.967368,30378.878909,59645.657315,166332.033815,86637.353521,17347.991885,57073.227540,83132.638939,100784.349249,661131.364176,...,50700.971161,67264.164871,40160.692701,29512.866999,54361.747225,298289.270979,310925.147466,30742.525710,28691.570854,2021-09-06
6,27539.428141,30400.718115,59529.307661,166090.584362,86268.563842,17366.899665,56940.022852,83207.587378,101230.507787,665225.112229,...,50433.413293,67217.021050,40001.052955,29679.360861,54734.226461,298346.760901,310698.423879,30843.403462,28361.043796,2021-09-07
7,27738.369159,30358.036150,59577.047635,166285.344620,86443.046789,17366.322826,56481.566031,83298.996386,100142.995192,665250.942317,...,50985.225947,67350.889301,40433.890977,29405.868598,55057.413500,297648.364661,307292.462941,30842.737634,28043.667539,2021-09-08
8,27914.857450,30206.399592,59243.862057,166335.805891,85960.176899,17153.740582,56577.151870,82972.940787,100028.689885,666273.433015,...,50522.701271,67640.945348,39987.541265,29485.390185,54763.208033,298562.497270,308892.947636,30795.278053,28418.314185,2021-09-09
9,27953.395191,30028.530143,59208.085288,165099.124616,85364.765081,17073.224948,56376.093569,82945.106106,99442.302346,663491.473802,...,50597.923101,67984.190035,39748.171154,29173.304845,54544.241289,298384.562908,308336.687130,30715.940625,28306.896816,2021-09-10


In [18]:
sub3.to_csv('stock.csv',index=False)

In [20]:
submission = pd.read_csv("sample_submission.csv")
submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-08-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-08-24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-08-25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-08-26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-08-27,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2021-09-06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2021-09-07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2021-09-08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2021-09-09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2021-09-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
submission.iloc[:,1:] = sub3.iloc[:,:]

In [25]:
submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-08-23,25777.856293,31795.723342,60446.774489,169651.903026,90793.351817,17786.718654,56560.870434,83965.562528,105691.642716,...,49024.860275,50002.806615,64430.343722,39155.000359,30199.152777,55741.235337,298386.019125,315536.342192,30780.812611,27985.201056
1,2021-08-24,25999.317065,31817.562548,60330.424835,169410.453573,90424.562138,17805.626435,56427.665746,84040.510966,106137.801255,...,48801.358415,49735.248747,64383.199900,38995.360613,30365.646639,56113.714573,298443.509048,315309.618605,30881.690363,27654.673997
2,2021-08-25,26198.258083,31774.880583,60378.164808,169605.213830,90599.045085,17805.049596,55969.208925,84131.919974,105050.288659,...,48898.949593,50287.061401,64517.068152,39428.198636,30092.154376,56436.901613,297745.112808,311903.657666,30881.024535,27337.297740
3,2021-08-26,26374.746375,31623.244025,60044.979230,169655.675101,90116.175195,17592.467352,56064.794764,83805.864375,104935.983352,...,48586.506320,49824.536724,64807.124198,38981.848924,30171.675963,56142.696145,298659.245417,313504.142362,30833.564954,27711.944386
4,2021-08-27,26413.284116,31445.374576,60009.202461,168418.993826,89520.763376,17511.951718,55863.736464,83778.029695,104349.595813,...,48755.649625,49899.758555,65150.368886,38742.478812,29859.590624,55923.729401,298481.311054,312947.881856,30754.227527,27600.527018
5,2021-09-06,27317.967368,30378.878909,59645.657315,166332.033815,86637.353521,17347.991885,57073.227540,83132.638939,100784.349249,...,49827.813833,50700.971161,67264.164871,40160.692701,29512.866999,54361.747225,298289.270979,310925.147466,30742.525710,28691.570854
6,2021-09-07,27539.428141,30400.718115,59529.307661,166090.584362,86268.563842,17366.899665,56940.022852,83207.587378,101230.507787,...,49604.311972,50433.413293,67217.021050,40001.052955,29679.360861,54734.226461,298346.760901,310698.423879,30843.403462,28361.043796
7,2021-09-08,27738.369159,30358.036150,59577.047635,166285.344620,86443.046789,17366.322826,56481.566031,83298.996386,100142.995192,...,49701.903150,50985.225947,67350.889301,40433.890977,29405.868598,55057.413500,297648.364661,307292.462941,30842.737634,28043.667539
8,2021-09-09,27914.857450,30206.399592,59243.862057,166335.805891,85960.176899,17153.740582,56577.151870,82972.940787,100028.689885,...,49389.459877,50522.701271,67640.945348,39987.541265,29485.390185,54763.208033,298562.497270,308892.947636,30795.278053,28418.314185
9,2021-09-10,27953.395191,30028.530143,59208.085288,165099.124616,85364.765081,17073.224948,56376.093569,82945.106106,99442.302346,...,49558.603182,50597.923101,67984.190035,39748.171154,29173.304845,54544.241289,298384.562908,308336.687130,30715.940625,28306.896816


In [26]:
submission.to_csv('stockkk.csv',index=False)